<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/StockFractionRobustness/62_Fraksi1_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data saham-saham fraksi 1 dijalankan menggunakan email shintafiaa@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



**Memasangkan Google Drive ke Google Colab**

* Kata kunci **from** digunakan untuk mengimpor hanya bagian tertentu dari modul (melakukan impor untuk bagian tertentu pada *library*)
* Modul merupakan file yang berisi sekumpulan fungsi yang ingin disertakan dalam aplikasi. Untuk membuat modul cukup simpan kode yang Anda inginkan dalam file dengan ekstensi file .py.
* **import** melakukan impor seluruh *library*.
* **Mounting** adalah proses membuat file dan direktori pada perangkat penyimpanan (seperti google drive) sehingga tersedia bagi pengguna untuk dapat diakses.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
    * [7.4. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)](#6.3)
      * [7.4.1. Agen 1: A2C](#6.4.1)
      * [7.4.2. Agen 2: PPO](#6.4.2)
      * [7.4.3. Agen 3: TD3](#6.4.3)
      * [7.4.4. Agen 4: SAC](#6.4.4)
      * [7.4.5. Agen 5: DDPG](#6.4.5)
    * [7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi](#6.4)
      * [7.5.1. Agen 1: A2C](#6.4.1)
      * [7.5.2. Agen 2: PPO](#6.4.2)
      * [7.5.3. Agen 3: TD3](#6.4.3)
      * [7.5.4. Agen 4: SAC](#6.4.4)
      * [7.5.5. Agen 5: DDPG](#6.4.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)
    * [8.2. Gambaran BackTesting](#7.2)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


* FinRL adalah *open-source framework* pertama yang menunjukkan potensi besar *Reinforcement Learning* keuangan.

* FinRL menyediakan berbagai pengaturan untuk melakukan *trading* dengan *Reinforcement Learning* seperti menyediakan ratusan pasar keuangan, algoritma yang canggih, berbagai macam aplikasi keuangan (alokasi portofolio, perdagangan mata uang kripto, *high-frequency trading*), *live trading, cloud deployment,* dll.

* Pengaplikasian pustaka FinRL dapat menggunakan alamat berikut, git+https://github.com/AI4Finance-Foundation/FinRL.git. Namun, untuk kemudahan pengaturan nilai sesuai kebutuhan, maka, dilakukanlah *forking* terhadap *repository* tersebut, lalu digunakanlah alamat berikut, git+https://github.com/shintaroudlotulhanafia/FinRL.git



In [2]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-siz0yooy
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-siz0yooy
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-auzpqim5/pyfolio_a62890ff5ddd4161b96b0adfd95904b0
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-auzpqim5/pyfolio_a62890ff5ddd4161b96b0adfd95904b0
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-auzpqim5/elegantrl_8811ecc1c9a54d2382e350d6e057b6e7
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-auzpqim5/elegantrl_8811ecc1c9a54d2382e350d6e057b6e7
     |████████████████████████████████| 2.3 MB 15.2 MB/s 
     |████████████████████████████████| 234 kB 87.6 MB/s 
     |█████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

* **config** berisi pengaturan tanggal periode training dan trading, indikator, hyperparameter setiap model/agen DRL.
* **config_tickers** berisi pengaturan daftar saham yang akan diproses.
* **import os** digunakan untuk meng-import modul, merupakan module pada python agar python itu sendiri berinteraksi langsung terhadap sistem operasi.
* **check_and_make_directories** digunakan untuk mengecek dan membuat folder.


In [4]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

-----
kelas YahooDownloader:
    Mengambil data saham harian dari
    Yahoo Finance API

    Atribut
    ----------
        start_date : str
            tanggal mulai data
        end_date : str
            tanggal akhir data
        ticker_list : list
            daftar ticker saham (dimodifikasi dari config.py)

    Methods
    -------
    fetch_data()


Penetapan tanggal training dan trading dapat dilakukan dengan mengaturnya pada finrl/config.py atau dengan menetapkannya di sel notebook.

In [5]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [6]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: <Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00 - Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00 - Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00 - Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']



In [7]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi1).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10916, 8)


* **df.shape** digunakan untuk mendapatkan jumlah baris dan kolom

In [8]:
df.shape

(10916, 8)

* **nunique()** mengembalikan jumlah nilai unik untuk setiap kolom. Dengan menentukan sumbu kolom ( axis='columns' ), metode nunique() mencari berdasarkan kolom tersebut dan mengembalikan jumlah nilai unik untuk setiap baris.

In [9]:
df.nunique()

date      2729
open      1014
high      1044
low       1021
close     1441
volume    8684
tic          4
day          5
dtype: int64

* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [10]:
print(df['tic'].unique())

['KIJA.JK' 'LCGP.JK' 'LMPI.JK' 'LPKR.JK']


* **describe()** digunakan untuk menampilkan detail statistik dasar seperti persentil, mean, std, dll. Hanya kolom yang bertipe numerik yang akan ditampilkan statistiknya.

In [11]:
df.describe()

,open,high,low,close,volume,day
count,10916.000000,10916.000000,10916.000000,10916.000000,1.091600e+04,10916.000000
mean,305.131510,310.550823,299.173490,299.052478,4.325543e+07,1.988641
std,238.405992,242.413365,233.786952,228.963016,7.793921e+07,1.409951
min,48.857891,48.857891,48.857891,48.236725,0.000000e+00,0.000000
25%,131.000000,135.000000,128.007675,130.000000,1.500000e+05,1.000000
50%,235.000000,240.000000,230.000000,235.000000,1.444527e+07,2.000000
75%,395.196686,400.751678,384.761902,379.425934,5.852571e+07,3.000000
max,1460.164551,1468.100220,1412.550415,1382.835327,1.637779e+09,4.000000


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan *ascending* atau *descending* dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [12]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,48.857891,49.835049,48.857891,48.236725,96630243,KIJA.JK,0
1,2009-01-05,50.000000,50.000000,50.000000,50.000000,0,LCGP.JK,0
2,2009-01-05,70.000000,70.000000,70.000000,70.000000,0,LMPI.JK,0
3,2009-01-05,596.003723,641.269836,596.003723,581.263184,32545542,LPKR.JK,0
4,2009-01-06,48.857891,52.766521,48.857891,48.236725,226697765,KIJA.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.



-----
kelas FeatureEngineer:
    Menyediakan metode untuk preprocessing data harga saham

    Attributes
    ----------
        use_technical_indicator : boolean
            menggunakan indikator teknis atau tidak
        tech_indicator_list : list
            daftar nama indikator teknis (dimodifikasi dari neofinrl_config.py)
        use_vix : boolean
            menggunakan Volatility Index (VIX) atau tidak
        use_turbulence : boolean
            menggunakan indeks turbulensi atau tidak
        user_defined_feature:boolean
            menggunakan fitur yang ditentukan pengguna atau tidak
    Methods
    -------
    preprocess_data()
        metode utama untuk melakukan feature engineering


In [13]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [14]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [15]:
processed.describe()

,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
count,10608.000000,10608.000000,10608.000000,10608.000000,1.060800e+04,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000
mean,305.551516,310.995315,299.558443,299.458119,4.353285e+07,2.019231,0.054662,50.356373,3.225393,18.179936,4.082293
std,238.745324,242.759425,234.098488,229.299719,7.774323e+07,1.397923,10.822138,9.970338,125.018907,7.335005,11.896582
min,48.857891,48.857891,48.857891,48.236725,0.000000e+00,0.000000,-80.906884,21.123967,-1000.000000,9.140000,0.000000
25%,131.000000,134.961945,128.005756,130.000000,1.550000e+05,1.000000,-2.768206,44.182374,-81.245442,13.287500,0.415048
50%,235.000000,240.000000,230.000000,235.000000,1.468182e+07,2.000000,-0.000153,49.056784,-10.579793,15.985000,1.401533
75%,396.783813,405.130127,384.761902,381.000305,5.901394e+07,3.000000,3.141874,55.150525,76.738162,20.629999,3.692559
max,1460.164551,1468.100220,1412.550415,1382.835327,1.637779e+09,4.000000,80.156574,100.000000,1000.000000,56.650002,364.268991


In [16]:
processed.nunique()

date           2652
open           1010
high           1038
low            1015
close          1434
volume         8472
tic               4
day               5
macd          10357
rsi_30         7459
cci_30         9825
vix            1441
turbulence     2399
dtype: int64

* **tolist()** digunakan untuk mengubah elemen data array menjadi *list*.
* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [17]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

* **date_range()** digunakan untuk mendapatkan frekuensi tetap DatetimeIndex. date_range() digunakan untuk membuat rentang tanggal di pandas.
* **max()** mengembalikan item dengan nilai tertinggi, atau item dengan nilai tertinggi dalam *iterable*. Jika nilainya adalah string, maka dilakukan perbandingan abjad.
* **min()** mengembalikan item dengan nilai terendah, atau item dengan nilai terendah dalam iterable. Jika nilainya adalah string,  maka dilakukan perbandingan abjad.
* **astype()** digunakan untuk mengubah tipe data dari suatu bentuk *series*.

In [18]:
list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

* **Array** dapat menyimpan elemen hanya dari satu tipe. Sedangkan **list** juga dapat menyimpan elemen dari tipe data yang berbeda.
* **list()** digunakan untuk membuat objek *list*. Objek *list* adalah kumpulan yang berurutan dan dapat diubah.
* **itertools.product()** digunakan untuk mencari produk kartesius dari iterator yang diberikan, outputnya adalah urutan leksikografis.

In [19]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'KIJA.JK'),
 ('2009-01-05', 'LCGP.JK'),
 ('2009-01-05', 'LMPI.JK'),
 ('2009-01-05', 'LPKR.JK'),
 ('2009-01-06', 'KIJA.JK'),
 ('2009-01-06', 'LCGP.JK'),
 ('2009-01-06', 'LMPI.JK'),
 ('2009-01-06', 'LPKR.JK'),
 ('2009-01-07', 'KIJA.JK'),
 ('2009-01-07', 'LCGP.JK'),
 ('2009-01-07', 'LMPI.JK'),
 ('2009-01-07', 'LPKR.JK'),
 ('2009-01-08', 'KIJA.JK'),
 ('2009-01-08', 'LCGP.JK'),
 ('2009-01-08', 'LMPI.JK'),
 ('2009-01-08', 'LPKR.JK'),
 ('2009-01-09', 'KIJA.JK'),
 ('2009-01-09', 'LCGP.JK'),
 ('2009-01-09', 'LMPI.JK'),
 ('2009-01-09', 'LPKR.JK'),
 ('2009-01-10', 'KIJA.JK'),
 ('2009-01-10', 'LCGP.JK'),
 ('2009-01-10', 'LMPI.JK'),
 ('2009-01-10', 'LPKR.JK'),
 ('2009-01-11', 'KIJA.JK'),
 ('2009-01-11', 'LCGP.JK'),
 ('2009-01-11', 'LMPI.JK'),
 ('2009-01-11', 'LPKR.JK'),
 ('2009-01-12', 'KIJA.JK'),
 ('2009-01-12', 'LCGP.JK'),
 ('2009-01-12', 'LMPI.JK'),
 ('2009-01-12', 'LPKR.JK'),
 ('2009-01-13', 'KIJA.JK'),
 ('2009-01-13', 'LCGP.JK'),
 ('2009-01-13', 'LMPI.JK'),
 ('2009-01-13', 'LPK

* **Pandas DataFrame** adalah dua dimensi yang dapat berubah ukuran, struktur data tabular yang berpotensi heterogen dengan sumbu berlabel (baris dan kolom).
* **merge()** berfungsi untuk memperbarui konten dua DataFrame dengan menggabungkannya bersama-sama, menggunakan metode yang ditentukan.

In [20]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999,2019-12-18,LPKR.JK,238.000000,240.000000,236.000000,238.000000,57097600.0,2.0,-1.505139,45.323461,-96.247961,12.580000,0.752572
16000,2019-12-19,KIJA.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16001,2019-12-19,LCGP.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16002,2019-12-19,LMPI.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* **isin()** digunakan untuk memfilter data frames. isin() dapat membantu untuk memilih baris dengan memiliki nilai tertentu (atau beberapa nilai tertentu) dalam kolom tertentu.

In [21]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


In [22]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


* **fillna()** digunakan untuk mengisi nilai NA/NaN menggunakan metode yang ditentukan. fillna(0) artinya baris NA/NaN diisi dengan nilai misalnya 0.

In [23]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


In [24]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
5,2009-01-06,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
6,2009-01-06,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
7,2009-01-06,LPKR.JK,626.181152,648.814209,626.181152,602.272766,53837632.0,1.0,0.471369,100.0,66.666667,38.560001,0.0
8,2009-01-07,KIJA.JK,48.857891,51.789364,48.857891,48.236725,203203608.0,2.0,0.000000,100.0,20.000000,43.389999,0.0
9,2009-01-07,LCGP.JK,50.000000,50.000000,50.000000,50.000000,10000.0,2.0,0.000000,100.0,20.000000,43.389999,0.0


In [25]:
processed_full.nunique()

date           2652
tic               4
open           1010
high           1038
low            1015
close          1434
volume         8472
day               5
macd          10357
rsi_30         7459
cci_30         9825
vix            1441
turbulence     2399
dtype: int64

* **info()** berfungsi untuk mencetak informasi tentang DataFrame. Informasi tersebut berisi jumlah kolom, label kolom, tipe data kolom, penggunaan memori, indeks rentang, dan jumlah sel di setiap kolom (nilai bukan nol).

In [26]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10608 entries, 0 to 15999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        10608 non-null  object 
 1   tic         10608 non-null  object 
 2   open        10608 non-null  float64
 3   high        10608 non-null  float64
 4   low         10608 non-null  float64
 5   close       10608 non-null  float64
 6   volume      10608 non-null  float64
 7   day         10608 non-null  float64
 8   macd        10608 non-null  float64
 9   rsi_30      10608 non-null  float64
 10  cci_30      10608 non-null  float64
 11  vix         10608 non-null  float64
 12  turbulence  10608 non-null  float64
dtypes: float64(11), object(2)
memory usage: 1.1+ MB


* **to_csv()** mengonversi DataFrame menjadi data CSV/comma separated value (nilai yang dipisahkan koma). Kita dapat melewatkan objek file untuk menulis data CSV ke dalam file. Jika tidak, data CSV dikembalikan dalam format string.
* **loc** berbasis label, yang berarti baris dan kolom diambil berdasarkan label baris dan kolomnya. **iloc** berbasis posisi integer, maka baris dan kolom dapat diambil dengan menggunakan nilai posisi integernya (posisi integer berbasis 0).

In [27]:
#Save the dataset by the tic filter
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  df_temp.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/DatasetsPerTic/Fraksi1/'+ stock +'.csv',index=False)
  display(df_temp)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
8,2009-01-07,KIJA.JK,48.857891,51.789364,48.857891,48.236725,203203608.0,2.0,0.000000,100.000000,20.000000,43.389999,0.000000
12,2009-01-08,KIJA.JK,48.857891,48.857891,48.857891,48.236725,7573494.0,3.0,0.000000,100.000000,-88.888889,42.560001,0.000000
16,2009-01-09,KIJA.JK,48.857891,49.835049,48.857891,48.236725,20502317.0,4.0,0.000000,100.000000,-39.215686,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,KIJA.JK,306.000000,306.000000,298.000000,300.000000,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.940000,3.498792
15976,2019-12-13,KIJA.JK,304.000000,304.000000,296.000000,302.000000,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.630000,3.309205
15988,2019-12-16,KIJA.JK,302.000000,306.000000,292.000000,298.000000,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.140000,5.699627
15992,2019-12-17,KIJA.JK,298.000000,300.000000,290.000000,298.000000,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.290000,0.008272


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
1,2009-01-05,LCGP.JK,50.0,50.0,50.0,50.0,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
5,2009-01-06,LCGP.JK,50.0,50.0,50.0,50.0,0.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
9,2009-01-07,LCGP.JK,50.0,50.0,50.0,50.0,10000.0,2.0,0.000000,100.000000,20.000000,43.389999,0.000000
13,2009-01-08,LCGP.JK,50.0,50.0,50.0,50.0,0.0,3.0,0.000000,100.000000,-88.888889,42.560001,0.000000
17,2009-01-09,LCGP.JK,50.0,50.0,50.0,50.0,10000.0,4.0,0.000000,100.000000,-39.215686,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,LCGP.JK,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.940000,3.498792
15977,2019-12-13,LCGP.JK,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.630000,3.309205
15989,2019-12-16,LCGP.JK,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.140000,5.699627
15993,2019-12-17,LCGP.JK,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.290000,0.008272


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2,2009-01-05,LMPI.JK,70.0,70.0,70.0,70.0,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
6,2009-01-06,LMPI.JK,70.0,70.0,70.0,70.0,0.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
10,2009-01-07,LMPI.JK,70.0,70.0,70.0,70.0,0.0,2.0,0.000000,100.000000,20.000000,43.389999,0.000000
14,2009-01-08,LMPI.JK,70.0,70.0,70.0,70.0,3.0,3.0,0.000000,100.000000,-88.888889,42.560001,0.000000
18,2009-01-09,LMPI.JK,70.0,70.0,70.0,70.0,0.0,4.0,0.000000,100.000000,-39.215686,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,LMPI.JK,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.940000,3.498792
15978,2019-12-13,LMPI.JK,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.630000,3.309205
15990,2019-12-16,LMPI.JK,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.140000,5.699627
15994,2019-12-17,LMPI.JK,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.290000,0.008272


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
7,2009-01-06,LPKR.JK,626.181152,648.814209,626.181152,602.272766,53837632.0,1.0,0.471369,100.000000,66.666667,38.560001,0.000000
11,2009-01-07,LPKR.JK,656.358521,671.447266,648.814209,609.275879,50325070.0,2.0,0.817784,100.000000,96.192728,43.389999,0.000000
15,2009-01-08,LPKR.JK,656.358521,663.902893,648.814209,609.275879,33617868.0,3.0,0.955298,100.000000,60.207117,42.560001,0.000000
19,2009-01-09,LPKR.JK,641.269836,656.358521,633.725464,595.269470,23678643.0,4.0,0.452563,64.566130,-2.515836,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,LPKR.JK,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.940000,3.498792
15979,2019-12-13,LPKR.JK,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.630000,3.309205
15991,2019-12-16,LPKR.JK,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.140000,5.699627
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272


In [28]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,4885.789108,4983.504868,4885.789108,4823.672485,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,5000.000000,5000.000000,5000.000000,5000.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,7000.000000,7000.000000,7000.000000,7000.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,59600.372314,64126.983643,59600.372314,58126.318359,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,4885.789108,5276.652145,4885.789108,4823.672485,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,24000.000000,24400.000000,23800.000000,24000.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,29800.000000,30600.000000,29200.000000,29600.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,11400.000000,11400.000000,11400.000000,11400.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,10100.000000,10100.000000,10100.000000,10100.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

* **data_split** membagi data menjadi himpunan *train, test,* dan/atau *validation.*
* **len()** mengembalikan jumlah item dalam suatu objek. Namun, jika objek adalah string, maka fungsi len() akan mengembalikan jumlah karakter dalam string.
* **tail()** digunakan untuk mendapatkan n baris terakhir.

In [29]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1980


In [30]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,LPKR.JK,38884.814453,39519.668579,38408.673096,38257.470703,58597272.0,3.0,-28.385327,27.868502,-102.555067,10.18,1.611104
2156,2017-12-29,KIJA.JK,28600.000000,28600.000000,28200.000000,28600.000000,16508800.0,4.0,-3.612968,34.278597,-97.415507,11.04,0.257610
2156,2017-12-29,LCGP.JK,7300.000000,8800.000000,7300.000000,8000.000000,8834300.0,4.0,-1.389227,43.985598,-8.855912,11.04,0.257610
2156,2017-12-29,LMPI.JK,16800.000000,17100.000000,16700.000000,16700.000000,35200.0,4.0,-1.008401,47.293358,-102.476931,11.04,0.257610
2156,2017-12-29,LPKR.JK,38884.814453,39202.243042,38567.388916,38414.910889,39843609.0,4.0,-27.095883,28.442145,-97.849420,11.04,0.257610


In [31]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363
0,2018-01-02,LCGP.JK,7900.000000,8300.000000,7900.000000,8100.000000,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363
0,2018-01-02,LMPI.JK,17100.000000,17200.000000,16700.000000,16800.000000,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363
0,2018-01-02,LPKR.JK,38884.814453,39043.527222,38408.673096,38100.030518,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363
1,2018-01-03,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529


In [32]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


Transaction Fee
    # BUY Fee => 0.23% (Broker Fee(0.19%) + Levy(0.04%))
    # SELL Fee => 0.54% (Broker Fee(0.29%) + Levy(0.04%) + PPN(0.11%) + PPh(0.1%))

In [33]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [34]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.



-----
kelas DRLAgent:
    Menyediakan implementasi untuk algoritma DRL

    Atribut
    ----------
        env: gym environment class
            kelas yang ditentukan pengguna

    Methods
    -------
        get_model()
            mengatur algoritma DRL
        train_model()
            melatih algoritma DRL dalam set data latih dan mengeluarkan model yang dilatih
        DRL_prediction()
            membuat prediksi dalam kumpulan data pengujian dan dapatkan hasil


In [35]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [36]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [37]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

--------------------------------------
| time/                 |            |
|    fps                | 348        |
|    iterations         | 100        |
|    time_elapsed       | 43         |
|    total_timesteps    | 15000      |
| train/                |            |
|    entropy_loss       | -5.72      |
|    explained_variance | -0.000259  |
|    learning_rate      | 0.0014     |
|    n_updates          | 99         |
|    policy_loss        | -1.61e+04  |
|    reward             | -255.58871 |
|    std                | 1.01       |
|    value_loss         | 2.42e+07   |
--------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 177920238.71
total_reward: 77920238.71
total_cost: 56996413.38
total_trades: 8156
Sharpe: 0.376
-------------------------------------
| time/                 |           |
|    fps                | 361       |
|    iterations         | 200       |
|    time_elapsed       | 82        |
|    total_timeste

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [38]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [39]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 86823415.03
total_reward: -13176584.97
total_cost: 57993383.08
total_trades: 8233
Sharpe: 0.076
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 84951139.92
total_reward: -15048860.08
total_cost: 61854336.03
total_trades: 8310
Sharpe: 0.063
-----------------------------------
| time/              |            |
|    fps             | 378        |
|    iterations      | 1          |
|    time_elapsed    | 132        |
|    total_timesteps | 50000      |
| train/             |            |
|    reward          | 124.030266 |
-----------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [40]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [41]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 125513990.65
total_reward: 25513990.65
total_cost: 21044727.21
total_trades: 3493
Sharpe: 0.302
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 97        |
|    time_elapsed    | 88        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 9.83e+07  |
|    critic_loss     | 8.64e+19  |
|    learning_rate   | 0.9       |
|    n_updates       | 6471      |
|    reward          | 20.227224 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 86        |
|    time_elapsed    | 200       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 7.52e+08  |
|    critic_loss     | 6.9e+15   |
|    learning_rate   | 0.9       |
|    n_updates       | 1509

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [42]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [43]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 70       |
|    time_elapsed    | 122      |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 2.02e+06 |
|    critic_loss     | 3.65e+07 |
|    ent_coef        | 1.25     |
|    learning_rate   | 0.0001   |
|    n_updates       | 8613     |
|    reward          | 0.0      |
---------------------------------
day: 2156, episode: 80
begin_total_asset: 100000000.00
end_total_asset: 100000000.00
total_reward: 0.00
total_cost: 0.00
total_trades: 0
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 70       |
|    time_elapsed    | 246      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 1.28e+06 |
|    critic_loss     | 1.71e+09 |
|    ent_coef        | 1.25     |
|    learning_rate   | 0.0001   |
|    n_update

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [44]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [45]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 154518534.75
total_reward: 54518534.75
total_cost: 229465.25
total_trades: 4119
Sharpe: 0.352
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 131      |
|    time_elapsed    | 65       |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 5.43e+04 |
|    critic_loss     | 1.74e+08 |
|    learning_rate   | 0.0003   |
|    n_updates       | 6471     |
|    reward          | 192.99   |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 116      |
|    time_elapsed    | 148      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 2.49e+04 |
|    critic_loss     | 1.63e+07 |
|    learning_rate   | 0.0003   |
|    n_updates       | 15099    |
|    reward       

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp10.000.000.

Pengaturan ambang turbulensi *(turbulence threshold)*

Atur ambang turbulensi menjadi lebih besar dari maksimum dalam data turbulensi sampel. Jika indeks turbulensi saat ini lebih besar dari ambang batas, maka dapat diasumsikan bahwa pasar pada periode tersebut sedang bergejolak

* **drop_duplicates()** menghapus baris duplikat.
* **quantile()** digunakan untuk menghitung kuantil nilai dalam sumbu tertentu. Sumbu default adalah baris.

In [46]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [47]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [48]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [49]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        4.121638
std        12.797609
min         0.000000
25%         0.391273
50%         1.345627
75%         3.532034
max       364.268991
Name: turbulence, dtype: float64

In [50]:
insample_risk_indicator.turbulence.quantile(0.996)

79.1987074499771

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [51]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [52]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363
0,2018-01-02,LCGP.JK,7900.000000,8300.000000,7900.000000,8100.000000,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363
0,2018-01-02,LMPI.JK,17100.000000,17200.000000,16700.000000,16800.000000,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363
0,2018-01-02,LPKR.JK,38884.814453,39043.527222,38408.673096,38100.030518,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363
1,2018-01-03,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529


<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [53]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


In [54]:
df_actions_and_account_value_a2c = pd.merge(df_actions_a2c, df_account_value_a2c, on='date', how='outer')

#adding new column
df_actions_and_account_value_a2c['total_shares'] = ""

#count the number of column of tics
numberOfColumns = len(df_actions_and_account_value_a2c.columns)
numberOfTic = numberOfColumns-2

#selecting column of actions in every tics
action_columns_a2c = df_actions_and_account_value_a2c.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_a2c['total_shares'] = action_columns_a2c.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_a2c['total_shares'] = df_actions_and_account_value_a2c['total_shares'].cumsum(skipna=False)

In [55]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

#swap points and rebounds columns
df_actions_and_account_value_a2c = swap_columns(df_actions_and_account_value_a2c, 'total_shares','account_value')

#Replace NaN value to 0
df_actions_and_account_value_a2c=df_actions_and_account_value_a2c.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_a2c)

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,2.0,100.0,0.0,0.0,102.0,1.000000e+08
1,2018-01-03,2.0,100.0,0.0,0.0,204.0,9.998801e+07
2,2018-01-04,2.0,100.0,0.0,0.0,306.0,9.996764e+07
3,2018-01-05,2.0,100.0,0.0,0.0,408.0,1.000557e+08
4,2018-01-08,2.0,100.0,0.0,0.0,510.0,1.000537e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,-100.0,0.0,20.0,9383.0,1.365833e+08
491,2019-12-13,0.0,78.0,0.0,-10.0,9451.0,1.365760e+08
492,2019-12-16,0.0,-100.0,0.0,20.0,9371.0,1.365727e+08
493,2019-12-17,0.0,78.0,0.0,-10.0,9439.0,1.365654e+08


In [56]:
df_actions_and_account_value_a2c.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi1/df_actions_and_account_value_a2c(4).csv',index=False)

<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [57]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


In [58]:
df_actions_and_account_value_ppo = pd.merge(df_actions_ppo, df_account_value_ppo, on='date', how='outer')

#adding new column
df_actions_and_account_value_ppo['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ppo = df_actions_and_account_value_ppo.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ppo['total_shares'] = action_columns_ppo.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ppo['total_shares'] = df_actions_and_account_value_ppo['total_shares'].cumsum(skipna=False)

In [59]:
#swap points and rebounds columns
df_actions_and_account_value_ppo = swap_columns(df_actions_and_account_value_ppo, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ppo=df_actions_and_account_value_ppo.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ppo)

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,0.0,0.0,0.0,0.0,0.0,100000000.0
1,2018-01-03,0.0,0.0,0.0,0.0,0.0,100000000.0
2,2018-01-04,0.0,0.0,0.0,0.0,0.0,100000000.0
3,2018-01-05,0.0,0.0,0.0,0.0,0.0,100000000.0
4,2018-01-08,0.0,0.0,0.0,0.0,0.0,100000000.0
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,0.0,100000000.0
491,2019-12-13,0.0,0.0,0.0,0.0,0.0,100000000.0
492,2019-12-16,0.0,0.0,0.0,0.0,0.0,100000000.0
493,2019-12-17,0.0,0.0,0.0,0.0,0.0,100000000.0


In [60]:
df_actions_and_account_value_ppo.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi1/df_actions_and_account_value_ppo(4).csv',index=False)

<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [61]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


In [62]:
df_actions_and_account_value_td3 = pd.merge(df_actions_td3, df_account_value_td3, on='date', how='outer')

#adding new column
df_actions_and_account_value_td3['total_shares'] = ""

#selecting column of actions in every tics
action_columns_td3 = df_actions_and_account_value_td3.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_td3['total_shares'] = action_columns_td3.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_td3['total_shares'] = df_actions_and_account_value_td3['total_shares'].cumsum(skipna=False)

In [63]:
#swap points and rebounds columns
df_actions_and_account_value_td3 = swap_columns(df_actions_and_account_value_td3, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_td3=df_actions_and_account_value_td3.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_td3)

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,100.0,100.0,100.0,100.0,400.0,1.000000e+08
1,2018-01-03,100.0,100.0,100.0,100.0,800.0,9.995323e+07
2,2018-01-04,100.0,100.0,100.0,100.0,1200.0,1.001982e+08
3,2018-01-05,100.0,100.0,100.0,100.0,1600.0,1.002070e+08
4,2018-01-08,100.0,100.0,100.0,100.0,2000.0,1.004376e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,4335.0,8.108172e+07
491,2019-12-13,0.0,0.0,0.0,0.0,4335.0,8.073852e+07
492,2019-12-16,0.0,0.0,0.0,0.0,4335.0,8.087492e+07
493,2019-12-17,0.0,0.0,0.0,0.0,4335.0,8.087492e+07


In [64]:
df_actions_and_account_value_td3.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi1/df_actions_and_account_value_td3(4).csv',index=False)

<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [65]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


In [66]:
df_actions_and_account_value_sac = pd.merge(df_actions_sac, df_account_value_sac, on='date', how='outer')

#adding new column
df_actions_and_account_value_sac['total_shares'] = ""

#selecting column of actions in every tics
action_columns_sac = df_actions_and_account_value_sac.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_sac['total_shares'] = action_columns_sac.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_sac['total_shares'] = df_actions_and_account_value_sac['total_shares'].cumsum(skipna=False)

In [67]:
#swap points and rebounds columns
df_actions_and_account_value_sac = swap_columns(df_actions_and_account_value_sac, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_sac=df_actions_and_account_value_sac.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_sac)

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,0.0,0.0,0.0,0.0,0.0,100000000.0
1,2018-01-03,0.0,0.0,0.0,0.0,0.0,100000000.0
2,2018-01-04,0.0,0.0,0.0,0.0,0.0,100000000.0
3,2018-01-05,0.0,0.0,0.0,0.0,0.0,100000000.0
4,2018-01-08,0.0,0.0,0.0,0.0,0.0,100000000.0
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,0.0,100000000.0
491,2019-12-13,0.0,0.0,0.0,0.0,0.0,100000000.0
492,2019-12-16,0.0,0.0,0.0,0.0,0.0,100000000.0
493,2019-12-17,0.0,0.0,0.0,0.0,0.0,100000000.0


In [68]:
df_actions_and_account_value_sac.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi1/df_actions_and_account_value_sac(3).csv',index=False)

<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [69]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


In [70]:
df_actions_and_account_value_ddpg = pd.merge(df_actions_ddpg, df_account_value_ddpg, on='date', how='outer')

#adding new column
df_actions_and_account_value_ddpg['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ddpg = df_actions_and_account_value_ddpg.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ddpg['total_shares'] = action_columns_ddpg.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ddpg['total_shares'] = df_actions_and_account_value_ddpg['total_shares'].cumsum(skipna=False)

In [71]:
#swap points and rebounds columns
df_actions_and_account_value_ddpg = swap_columns(df_actions_and_account_value_ddpg, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ddpg=df_actions_and_account_value_ddpg.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ddpg)

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,0.0,100.0,0.0,0.0,100.0,1.000000e+08
1,2018-01-03,0.0,100.0,0.0,0.0,200.0,9.998814e+07
2,2018-01-04,0.0,100.0,0.0,0.0,300.0,9.996630e+07
3,2018-01-05,0.0,100.0,0.0,0.0,400.0,1.000545e+08
4,2018-01-08,0.0,100.0,0.0,0.0,500.0,1.000526e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,12534.0,1.428917e+08
491,2019-12-13,0.0,0.0,0.0,0.0,12534.0,1.428917e+08
492,2019-12-16,0.0,0.0,0.0,0.0,12534.0,1.428917e+08
493,2019-12-17,0.0,0.0,0.0,0.0,12534.0,1.428917e+08


In [72]:
df_actions_and_account_value_ddpg.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi1/df_actions_and_account_value_ddpg(4).csv',index=False)

##7.4 Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)

###7.4.1. Agen 1: A2C

In [73]:
#Replace NaN value to 0
action_columns_a2c=action_columns_a2c.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_a2c)

j=0
for j in range(numberOfRows):
  for i in action_columns_a2c:
   val = action_columns_a2c[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_a2c, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi1/df_actions_and_account_str_value_a2c(4).csv',index=False)
df_actions_str

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,Beli 2.0 saham,Beli 100.0 saham,Tahan,Tahan,102.0,1.000000e+08
1,2018-01-03,Beli 2.0 saham,Beli 100.0 saham,Tahan,Tahan,204.0,9.998801e+07
2,2018-01-04,Beli 2.0 saham,Beli 100.0 saham,Tahan,Tahan,306.0,9.996764e+07
3,2018-01-05,Beli 2.0 saham,Beli 100.0 saham,Tahan,Tahan,408.0,1.000557e+08
4,2018-01-08,Beli 2.0 saham,Beli 100.0 saham,Tahan,Tahan,510.0,1.000537e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Jual -100.0 saham,Tahan,Beli 20.0 saham,9383.0,1.365833e+08
491,2019-12-13,Tahan,Beli 78.0 saham,Tahan,Jual -10.0 saham,9451.0,1.365760e+08
492,2019-12-16,Tahan,Jual -100.0 saham,Tahan,Beli 20.0 saham,9371.0,1.365727e+08
493,2019-12-17,Tahan,Beli 78.0 saham,Tahan,Jual -10.0 saham,9439.0,1.365654e+08


###7.4.2. Agen 2: PPO

In [74]:
#Replace NaN value to 0
action_columns_ppo=action_columns_ppo.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ppo)

j=0
for j in range(numberOfRows):
  for i in action_columns_ppo:
   val = action_columns_ppo[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ppo, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi1/df_actions_and_account_str_value_ppo(4).csv',index=False)
df_actions_str

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
1,2018-01-03,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0


###7.4.1. Agen 3: TD3

In [75]:
#Replace NaN value to 0
action_columns_td3=action_columns_td3.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_td3)

j=0
for j in range(numberOfRows):
  for i in action_columns_td3:
   val = action_columns_td3[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_td3, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi1/df_actions_and_account_str_value_td3(4).csv',index=False)
df_actions_str

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,400.0,1.000000e+08
1,2018-01-03,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,800.0,9.995323e+07
2,2018-01-04,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,1200.0,1.001982e+08
3,2018-01-05,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,1600.0,1.002070e+08
4,2018-01-08,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,2000.0,1.004376e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,4335.0,8.108172e+07
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,4335.0,8.073852e+07
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,4335.0,8.087492e+07
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,4335.0,8.087492e+07


###7.4.1. Agen 4: SAC

In [76]:
#Replace NaN value to 0
action_columns_sac=action_columns_sac.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_sac)

j=0
for j in range(numberOfRows):
  for i in action_columns_sac:
   val = action_columns_sac[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_sac, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi1/df_actions_and_account_str_value_sac(4).csv',index=False)
df_actions_str

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
1,2018-01-03,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0


###7.4.1. Agen 5: DDPG

In [77]:
#Replace NaN value to 0
action_columns_ddpg=action_columns_ddpg.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ddpg)

j=0
for j in range(numberOfRows):
  for i in action_columns_ddpg:
   val = action_columns_ddpg[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ddpg, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi1/df_actions_and_account_str_value_ddpg(4).csv',index=False)
df_actions_str

,date,KIJA.JK,LCGP.JK,LMPI.JK,LPKR.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 100.0 saham,Tahan,Tahan,100.0,1.000000e+08
1,2018-01-03,Tahan,Beli 100.0 saham,Tahan,Tahan,200.0,9.998814e+07
2,2018-01-04,Tahan,Beli 100.0 saham,Tahan,Tahan,300.0,9.996630e+07
3,2018-01-05,Tahan,Beli 100.0 saham,Tahan,Tahan,400.0,1.000545e+08
4,2018-01-08,Tahan,Beli 100.0 saham,Tahan,Tahan,500.0,1.000526e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,12534.0,1.428917e+08
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,12534.0,1.428917e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,12534.0,1.428917e+08
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,12534.0,1.428917e+08


<a id='6.4'></a>
##7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi

<a id='6.4.1'></a>
###7.5.1. Agen 1: A2C

In [78]:
#drop some columns
df_actions_and_account_value_a2c = df_actions_and_account_value_a2c.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_a2c:
    df_temp = df_actions_and_account_value_a2c[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic-1:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi1/1_A2C/df_states_and_actions_a2c_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,KIJA.JK,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,2.0
13140,2018-01-03,KIJA.JK,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,2.0
13144,2018-01-04,KIJA.JK,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,2.0
13148,2018-01-05,KIJA.JK,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,2.0
13160,2018-01-08,KIJA.JK,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,KIJA.JK,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,0.0
15976,2019-12-13,KIJA.JK,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,0.0
15988,2019-12-16,KIJA.JK,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,0.0
15992,2019-12-17,KIJA.JK,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,LCGP.JK,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,100.0
13141,2018-01-03,LCGP.JK,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,100.0
13145,2018-01-04,LCGP.JK,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,100.0
13149,2018-01-05,LCGP.JK,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,100.0
13161,2018-01-08,LCGP.JK,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,LCGP.JK,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,-100.0
15977,2019-12-13,LCGP.JK,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,78.0
15989,2019-12-16,LCGP.JK,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,-100.0
15993,2019-12-17,LCGP.JK,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,78.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,LMPI.JK,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,0.0
13142,2018-01-03,LMPI.JK,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,0.0
13146,2018-01-04,LMPI.JK,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,0.0
13150,2018-01-05,LMPI.JK,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,0.0
13162,2018-01-08,LMPI.JK,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,LMPI.JK,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,0.0
15978,2019-12-13,LMPI.JK,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,0.0
15990,2019-12-16,LMPI.JK,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,0.0
15994,2019-12-17,LMPI.JK,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,LPKR.JK,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,0.0
13143,2018-01-03,LPKR.JK,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,0.0
13147,2018-01-04,LPKR.JK,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,0.0
13151,2018-01-05,LPKR.JK,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,0.0
13163,2018-01-08,LPKR.JK,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,LPKR.JK,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,20.0
15979,2019-12-13,LPKR.JK,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,-10.0
15991,2019-12-16,LPKR.JK,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,20.0
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,-10.0


####String Action Version

In [79]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  #i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi1/1_A2C/df_states_and_actions_str_a2c_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,KIJA.JK
0,13136,2018-01-02,KIJA.JK,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,2.0,Beli 2.0 saham
1,13140,2018-01-03,KIJA.JK,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,2.0,Beli 2.0 saham
2,13144,2018-01-04,KIJA.JK,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,2.0,Beli 2.0 saham
3,13148,2018-01-05,KIJA.JK,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,2.0,Beli 2.0 saham
4,13160,2018-01-08,KIJA.JK,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,2.0,Beli 2.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,KIJA.JK,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,0.0,Tahan
491,15976,2019-12-13,KIJA.JK,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,0.0,Tahan
492,15988,2019-12-16,KIJA.JK,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,0.0,Tahan
493,15992,2019-12-17,KIJA.JK,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,LCGP.JK
0,13137,2018-01-02,LCGP.JK,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,100.0,Beli 100.0 saham
1,13141,2018-01-03,LCGP.JK,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,100.0,Beli 100.0 saham
2,13145,2018-01-04,LCGP.JK,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,100.0,Beli 100.0 saham
3,13149,2018-01-05,LCGP.JK,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,100.0,Beli 100.0 saham
4,13161,2018-01-08,LCGP.JK,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,100.0,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,LCGP.JK,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,-100.0,Tahan
491,15977,2019-12-13,LCGP.JK,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,78.0,Beli 78.0 saham
492,15989,2019-12-16,LCGP.JK,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,-100.0,Tahan
493,15993,2019-12-17,LCGP.JK,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,78.0,Beli 78.0 saham


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,LMPI.JK
0,13138,2018-01-02,LMPI.JK,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,0.0,Tahan
1,13142,2018-01-03,LMPI.JK,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,0.0,Tahan
2,13146,2018-01-04,LMPI.JK,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,0.0,Tahan
3,13150,2018-01-05,LMPI.JK,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,0.0,Tahan
4,13162,2018-01-08,LMPI.JK,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,LMPI.JK,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,0.0,Tahan
491,15978,2019-12-13,LMPI.JK,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,0.0,Tahan
492,15990,2019-12-16,LMPI.JK,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,0.0,Tahan
493,15994,2019-12-17,LMPI.JK,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,LPKR.JK
0,13139,2018-01-02,LPKR.JK,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,0.0,Tahan
1,13143,2018-01-03,LPKR.JK,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,0.0,Tahan
2,13147,2018-01-04,LPKR.JK,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,0.0,Tahan
3,13151,2018-01-05,LPKR.JK,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,0.0,Tahan
4,13163,2018-01-08,LPKR.JK,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,LPKR.JK,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,20.0,Beli 20.0 saham
491,15979,2019-12-13,LPKR.JK,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,-10.0,Tahan
492,15991,2019-12-16,LPKR.JK,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,20.0,Beli 20.0 saham
493,15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,-10.0,Tahan


<a id='6.4.2'></a>
###7.5.2.Agen 2: PPO

In [80]:
#drop some columns
df_actions_and_account_value_ppo = df_actions_and_account_value_ppo.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ppo:
    df_temp = df_actions_and_account_value_ppo[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi1/2_PPO/df_states_and_actions_ppo_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,KIJA.JK,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,0.0
13140,2018-01-03,KIJA.JK,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,0.0
13144,2018-01-04,KIJA.JK,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,0.0
13148,2018-01-05,KIJA.JK,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,0.0
13160,2018-01-08,KIJA.JK,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,KIJA.JK,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,0.0
15976,2019-12-13,KIJA.JK,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,0.0
15988,2019-12-16,KIJA.JK,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,0.0
15992,2019-12-17,KIJA.JK,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,LCGP.JK,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,0.0
13141,2018-01-03,LCGP.JK,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,0.0
13145,2018-01-04,LCGP.JK,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,0.0
13149,2018-01-05,LCGP.JK,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,0.0
13161,2018-01-08,LCGP.JK,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,LCGP.JK,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,0.0
15977,2019-12-13,LCGP.JK,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,0.0
15989,2019-12-16,LCGP.JK,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,0.0
15993,2019-12-17,LCGP.JK,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,LMPI.JK,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,0.0
13142,2018-01-03,LMPI.JK,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,0.0
13146,2018-01-04,LMPI.JK,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,0.0
13150,2018-01-05,LMPI.JK,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,0.0
13162,2018-01-08,LMPI.JK,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,LMPI.JK,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,0.0
15978,2019-12-13,LMPI.JK,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,0.0
15990,2019-12-16,LMPI.JK,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,0.0
15994,2019-12-17,LMPI.JK,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,LPKR.JK,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,0.0
13143,2018-01-03,LPKR.JK,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,0.0
13147,2018-01-04,LPKR.JK,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,0.0
13151,2018-01-05,LPKR.JK,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,0.0
13163,2018-01-08,LPKR.JK,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,LPKR.JK,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,0.0
15979,2019-12-13,LPKR.JK,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,0.0
15991,2019-12-16,LPKR.JK,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,0.0
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,0.0


####String Action Version

In [81]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi1/2_PPO/df_states_and_actions_str_ppo_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,KIJA.JK
0,13136,2018-01-02,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,Tahan
1,13140,2018-01-03,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,Tahan
2,13144,2018-01-04,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,Tahan
3,13148,2018-01-05,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,Tahan
4,13160,2018-01-08,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,Tahan
491,15976,2019-12-13,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,Tahan
492,15988,2019-12-16,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,Tahan
493,15992,2019-12-17,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LCGP.JK
0,13137,2018-01-02,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,Tahan
1,13141,2018-01-03,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,Tahan
2,13145,2018-01-04,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,Tahan
3,13149,2018-01-05,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,Tahan
4,13161,2018-01-08,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,Tahan
491,15977,2019-12-13,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,Tahan
492,15989,2019-12-16,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,Tahan
493,15993,2019-12-17,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LMPI.JK
0,13138,2018-01-02,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,Tahan
1,13142,2018-01-03,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,Tahan
2,13146,2018-01-04,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,Tahan
3,13150,2018-01-05,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,Tahan
4,13162,2018-01-08,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,Tahan
491,15978,2019-12-13,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,Tahan
492,15990,2019-12-16,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,Tahan
493,15994,2019-12-17,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LPKR.JK
0,13139,2018-01-02,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,Tahan
1,13143,2018-01-03,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,Tahan
2,13147,2018-01-04,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,Tahan
3,13151,2018-01-05,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,Tahan
4,13163,2018-01-08,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,Tahan
491,15979,2019-12-13,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,Tahan
492,15991,2019-12-16,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,Tahan
493,15995,2019-12-17,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,Tahan


<a id='6.4.3'></a>
###7.5.3. Agen 3: TD3

In [82]:
#drop some columns
df_actions_and_account_value_td3 = df_actions_and_account_value_td3.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_td3:
    df_temp = df_actions_and_account_value_td3[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi1/3_TD3/df_states_and_actions_td3_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,KIJA.JK,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,100.0
13140,2018-01-03,KIJA.JK,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,100.0
13144,2018-01-04,KIJA.JK,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,100.0
13148,2018-01-05,KIJA.JK,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,100.0
13160,2018-01-08,KIJA.JK,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,KIJA.JK,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,0.0
15976,2019-12-13,KIJA.JK,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,0.0
15988,2019-12-16,KIJA.JK,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,0.0
15992,2019-12-17,KIJA.JK,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,LCGP.JK,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,100.0
13141,2018-01-03,LCGP.JK,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,100.0
13145,2018-01-04,LCGP.JK,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,100.0
13149,2018-01-05,LCGP.JK,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,100.0
13161,2018-01-08,LCGP.JK,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,LCGP.JK,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,0.0
15977,2019-12-13,LCGP.JK,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,0.0
15989,2019-12-16,LCGP.JK,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,0.0
15993,2019-12-17,LCGP.JK,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,LMPI.JK,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,100.0
13142,2018-01-03,LMPI.JK,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,100.0
13146,2018-01-04,LMPI.JK,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,100.0
13150,2018-01-05,LMPI.JK,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,100.0
13162,2018-01-08,LMPI.JK,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,LMPI.JK,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,0.0
15978,2019-12-13,LMPI.JK,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,0.0
15990,2019-12-16,LMPI.JK,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,0.0
15994,2019-12-17,LMPI.JK,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,LPKR.JK,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,100.0
13143,2018-01-03,LPKR.JK,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,100.0
13147,2018-01-04,LPKR.JK,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,100.0
13151,2018-01-05,LPKR.JK,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,100.0
13163,2018-01-08,LPKR.JK,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,LPKR.JK,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,0.0
15979,2019-12-13,LPKR.JK,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,0.0
15991,2019-12-16,LPKR.JK,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,0.0
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,0.0


####String Action Version

In [83]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi1/3_TD3/df_states_and_actions_str_td3_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,KIJA.JK
0,13136,2018-01-02,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,Beli 100.0 saham
1,13140,2018-01-03,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,Beli 100.0 saham
2,13144,2018-01-04,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,Beli 100.0 saham
3,13148,2018-01-05,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,Beli 100.0 saham
4,13160,2018-01-08,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,Tahan
491,15976,2019-12-13,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,Tahan
492,15988,2019-12-16,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,Tahan
493,15992,2019-12-17,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LCGP.JK
0,13137,2018-01-02,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,Beli 100.0 saham
1,13141,2018-01-03,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,Beli 100.0 saham
2,13145,2018-01-04,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,Beli 100.0 saham
3,13149,2018-01-05,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,Beli 100.0 saham
4,13161,2018-01-08,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,Tahan
491,15977,2019-12-13,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,Tahan
492,15989,2019-12-16,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,Tahan
493,15993,2019-12-17,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LMPI.JK
0,13138,2018-01-02,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,Beli 100.0 saham
1,13142,2018-01-03,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,Beli 100.0 saham
2,13146,2018-01-04,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,Beli 100.0 saham
3,13150,2018-01-05,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,Beli 100.0 saham
4,13162,2018-01-08,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,Tahan
491,15978,2019-12-13,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,Tahan
492,15990,2019-12-16,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,Tahan
493,15994,2019-12-17,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LPKR.JK
0,13139,2018-01-02,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,Beli 100.0 saham
1,13143,2018-01-03,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,Beli 100.0 saham
2,13147,2018-01-04,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,Beli 100.0 saham
3,13151,2018-01-05,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,Beli 100.0 saham
4,13163,2018-01-08,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,Tahan
491,15979,2019-12-13,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,Tahan
492,15991,2019-12-16,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,Tahan
493,15995,2019-12-17,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,Tahan


<a id='6.4.4'></a>
###7.5.4. Agen 4: SAC

In [84]:
#drop some columns
df_actions_and_account_value_sac = df_actions_and_account_value_sac.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_sac:
    df_temp = df_actions_and_account_value_sac[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi1/4_SAC/df_states_and_actions_sac_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,KIJA.JK,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,0.0
13140,2018-01-03,KIJA.JK,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,0.0
13144,2018-01-04,KIJA.JK,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,0.0
13148,2018-01-05,KIJA.JK,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,0.0
13160,2018-01-08,KIJA.JK,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,KIJA.JK,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,0.0
15976,2019-12-13,KIJA.JK,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,0.0
15988,2019-12-16,KIJA.JK,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,0.0
15992,2019-12-17,KIJA.JK,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,LCGP.JK,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,0.0
13141,2018-01-03,LCGP.JK,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,0.0
13145,2018-01-04,LCGP.JK,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,0.0
13149,2018-01-05,LCGP.JK,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,0.0
13161,2018-01-08,LCGP.JK,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,LCGP.JK,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,0.0
15977,2019-12-13,LCGP.JK,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,0.0
15989,2019-12-16,LCGP.JK,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,0.0
15993,2019-12-17,LCGP.JK,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,LMPI.JK,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,0.0
13142,2018-01-03,LMPI.JK,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,0.0
13146,2018-01-04,LMPI.JK,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,0.0
13150,2018-01-05,LMPI.JK,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,0.0
13162,2018-01-08,LMPI.JK,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,LMPI.JK,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,0.0
15978,2019-12-13,LMPI.JK,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,0.0
15990,2019-12-16,LMPI.JK,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,0.0
15994,2019-12-17,LMPI.JK,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,LPKR.JK,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,0.0
13143,2018-01-03,LPKR.JK,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,0.0
13147,2018-01-04,LPKR.JK,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,0.0
13151,2018-01-05,LPKR.JK,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,0.0
13163,2018-01-08,LPKR.JK,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,LPKR.JK,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,0.0
15979,2019-12-13,LPKR.JK,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,0.0
15991,2019-12-16,LPKR.JK,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,0.0
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,0.0


####String Action Version

In [85]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi1/4_SAC/df_states_and_actions_str_sac_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,KIJA.JK
0,13136,2018-01-02,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,Tahan
1,13140,2018-01-03,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,Tahan
2,13144,2018-01-04,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,Tahan
3,13148,2018-01-05,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,Tahan
4,13160,2018-01-08,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,Tahan
491,15976,2019-12-13,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,Tahan
492,15988,2019-12-16,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,Tahan
493,15992,2019-12-17,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LCGP.JK
0,13137,2018-01-02,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,Tahan
1,13141,2018-01-03,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,Tahan
2,13145,2018-01-04,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,Tahan
3,13149,2018-01-05,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,Tahan
4,13161,2018-01-08,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,Tahan
491,15977,2019-12-13,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,Tahan
492,15989,2019-12-16,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,Tahan
493,15993,2019-12-17,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LMPI.JK
0,13138,2018-01-02,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,Tahan
1,13142,2018-01-03,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,Tahan
2,13146,2018-01-04,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,Tahan
3,13150,2018-01-05,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,Tahan
4,13162,2018-01-08,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,Tahan
491,15978,2019-12-13,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,Tahan
492,15990,2019-12-16,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,Tahan
493,15994,2019-12-17,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LPKR.JK
0,13139,2018-01-02,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,Tahan
1,13143,2018-01-03,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,Tahan
2,13147,2018-01-04,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,Tahan
3,13151,2018-01-05,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,Tahan
4,13163,2018-01-08,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,Tahan
491,15979,2019-12-13,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,Tahan
492,15991,2019-12-16,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,Tahan
493,15995,2019-12-17,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,Tahan


<a id='6.5.5'></a>
###7.4.5. Agen 5: DDPG

In [86]:
#drop some columns
df_actions_and_account_value_ddpg = df_actions_and_account_value_ddpg.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ddpg:
    df_temp = df_actions_and_account_value_ddpg[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi1/5_DDPG/df_states_and_actions_ddpg_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,KIJA.JK,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,0.0
13140,2018-01-03,KIJA.JK,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,0.0
13144,2018-01-04,KIJA.JK,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,0.0
13148,2018-01-05,KIJA.JK,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,0.0
13160,2018-01-08,KIJA.JK,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,KIJA.JK,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,0.0
15976,2019-12-13,KIJA.JK,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,0.0
15988,2019-12-16,KIJA.JK,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,0.0
15992,2019-12-17,KIJA.JK,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,LCGP.JK,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,100.0
13141,2018-01-03,LCGP.JK,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,100.0
13145,2018-01-04,LCGP.JK,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,100.0
13149,2018-01-05,LCGP.JK,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,100.0
13161,2018-01-08,LCGP.JK,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,LCGP.JK,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,0.0
15977,2019-12-13,LCGP.JK,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,0.0
15989,2019-12-16,LCGP.JK,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,0.0
15993,2019-12-17,LCGP.JK,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,LMPI.JK,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,0.0
13142,2018-01-03,LMPI.JK,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,0.0
13146,2018-01-04,LMPI.JK,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,0.0
13150,2018-01-05,LMPI.JK,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,0.0
13162,2018-01-08,LMPI.JK,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,LMPI.JK,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,0.0
15978,2019-12-13,LMPI.JK,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,0.0
15990,2019-12-16,LMPI.JK,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,0.0
15994,2019-12-17,LMPI.JK,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,LPKR.JK,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,0.0
13143,2018-01-03,LPKR.JK,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,0.0
13147,2018-01-04,LPKR.JK,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,0.0
13151,2018-01-05,LPKR.JK,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,0.0
13163,2018-01-08,LPKR.JK,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,LPKR.JK,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,0.0
15979,2019-12-13,LPKR.JK,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,0.0
15991,2019-12-16,LPKR.JK,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,0.0
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,0.0


####String Action Version

In [87]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi1/5_DDPG/df_states_and_actions_str_ddpg_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,KIJA.JK
0,13136,2018-01-02,286.0,286.0,280.0,284.0,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363,Tahan
1,13140,2018-01-03,286.0,286.0,280.0,284.0,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529,Tahan
2,13144,2018-01-04,284.0,288.0,280.0,288.0,9226600.0,3.0,-3.091606,39.392343,-76.201201,9.22,4.255984,Tahan
3,13148,2018-01-05,286.0,288.0,282.0,288.0,5275800.0,4.0,-2.758949,39.392343,-62.124248,9.22,1.150439,Tahan
4,13160,2018-01-08,288.0,290.0,284.0,288.0,9320700.0,0.0,-2.466879,39.392343,-34.953112,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,306.0,306.0,298.0,300.0,6815600.0,3.0,-3.666153,35.232325,-214.285714,13.94,3.498792,Tahan
491,15976,2019-12-13,304.0,304.0,296.0,302.0,7740600.0,4.0,-4.046407,37.776953,-196.138780,12.63,3.309205,Tahan
492,15988,2019-12-16,302.0,306.0,292.0,298.0,11035000.0,0.0,-4.617301,34.937038,-197.478992,12.14,5.699627,Tahan
493,15992,2019-12-17,298.0,300.0,290.0,298.0,8484600.0,1.0,-5.011965,34.937038,-200.823643,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LCGP.JK
0,13137,2018-01-02,79.0,83.0,79.0,81.0,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363,Beli 100.0 saham
1,13141,2018-01-03,84.0,84.0,79.0,80.0,2184900.0,2.0,-1.011444,44.145508,5.501939,9.15,0.089529,Beli 100.0 saham
2,13145,2018-01-04,80.0,81.0,75.0,79.0,4071200.0,3.0,-1.019024,43.438363,-12.264151,9.22,4.255984,Beli 100.0 saham
3,13149,2018-01-05,74.0,82.0,74.0,82.0,3361000.0,4.0,-0.774033,46.117026,0.298775,9.22,1.150439,Beli 100.0 saham
4,13161,2018-01-08,82.0,84.0,80.0,82.0,5460500.0,0.0,-0.573269,46.117026,28.696195,9.52,0.837700,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,114.0,114.0,114.0,114.0,0.0,3.0,-0.000006,44.937077,-214.285714,13.94,3.498792,Tahan
491,15977,2019-12-13,114.0,114.0,114.0,114.0,0.0,4.0,-0.000005,44.937077,-196.138780,12.63,3.309205,Tahan
492,15989,2019-12-16,114.0,114.0,114.0,114.0,0.0,0.0,-0.000005,44.937077,-197.478992,12.14,5.699627,Tahan
493,15993,2019-12-17,114.0,114.0,114.0,114.0,0.0,1.0,-0.000004,44.937077,-200.823643,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LMPI.JK
0,13138,2018-01-02,171.0,172.0,167.0,168.0,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363,Tahan
1,13142,2018-01-03,167.0,169.0,167.0,168.0,52300.0,2.0,-1.074350,48.158048,-111.457036,9.15,0.089529,Tahan
2,13146,2018-01-04,168.0,172.0,167.0,172.0,13800.0,3.0,-0.725298,51.559839,36.189069,9.22,4.255984,Tahan
3,13150,2018-01-05,169.0,170.0,167.0,170.0,22800.0,4.0,-0.603102,49.867313,-48.689139,9.22,1.150439,Tahan
4,13162,2018-01-08,171.0,171.0,170.0,170.0,1500.0,0.0,-0.500492,49.867313,41.198502,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,98.0,101.0,94.0,101.0,1500.0,3.0,-6.670796,39.194438,-75.537076,13.94,3.498792,Tahan
491,15978,2019-12-13,101.0,101.0,96.0,96.0,400.0,4.0,-6.516368,36.971532,-78.759715,12.63,3.309205,Tahan
492,15990,2019-12-16,96.0,101.0,91.0,101.0,154400.0,0.0,-5.922256,40.464511,-75.457317,12.14,5.699627,Tahan
493,15994,2019-12-17,101.0,101.0,101.0,101.0,100.0,1.0,-5.389293,40.464511,-46.990932,12.29,0.008272,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,LPKR.JK
0,13139,2018-01-02,388.848145,390.435272,384.086731,381.000305,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363,Tahan
1,13143,2018-01-03,384.086731,387.261017,382.499603,379.425934,37161292.0,2.0,-23.701934,27.726110,-86.243939,9.15,0.089529,Tahan
2,13147,2018-01-04,385.673889,388.848145,382.499603,385.723480,31210067.0,3.0,-22.146830,30.204775,-75.256719,9.22,4.255984,Tahan
3,13151,2018-01-05,388.848145,392.022430,387.261017,385.723480,18278718.0,4.0,-20.676058,30.204775,-64.636790,9.22,1.150439,Tahan
4,13163,2018-01-08,390.435272,400.751678,388.848145,392.020996,43576876.0,0.0,-18.785753,32.675662,-47.912265,9.52,0.837700,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,242.000000,242.000000,236.000000,240.000000,50143300.0,3.0,-0.990862,46.675013,-55.225924,13.94,3.498792,Tahan
491,15979,2019-12-13,240.000000,242.000000,236.000000,240.000000,84992200.0,4.0,-1.138868,46.675013,-60.275273,12.63,3.309205,Tahan
492,15991,2019-12-16,240.000000,240.000000,238.000000,240.000000,73414400.0,0.0,-1.241849,46.675013,-65.743945,12.14,5.699627,Tahan
493,15995,2019-12-17,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.29,0.008272,Tahan


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [88]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return          0.169903
Cumulative returns     0.361025
Annual volatility      0.328029
Sharpe ratio           0.640691
Calmar ratio           0.744679
Stability              0.455344
Max drawdown          -0.228157
Omega ratio            1.161726
Sortino ratio          1.076781
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.265326
Daily value at risk   -0.040494
dtype: float64


###8.1.2 Agen ppo

In [89]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return          0.0
Cumulative returns     0.0
Annual volatility      0.0
Sharpe ratio           NaN
Calmar ratio           NaN
Stability              0.0
Max drawdown           0.0
Omega ratio            NaN
Sortino ratio          NaN
Skew                   NaN
Kurtosis               NaN
Tail ratio             NaN
Daily value at risk    0.0
dtype: float64


/usr/local/lib/python3.7/dist-packages/empyrical/stats.py:713: RuntimeWarning: invalid value encountered in true_divide
  out=out,
/usr/local/lib/python3.7/dist-packages/empyrical/stats.py:799: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
/usr/local/lib/python3.7/dist-packages/empyrical/stats.py:1528: RuntimeWarning: invalid value encountered in double_scalars
  np.abs(np.percentile(returns, 5))


###8.1.3 Agen TD3

In [90]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return         -0.104843
Cumulative returns    -0.195519
Annual volatility      0.175517
Sharpe ratio          -0.544496
Calmar ratio          -0.344796
Stability              0.160136
Max drawdown          -0.304074
Omega ratio            0.907430
Sortino ratio         -0.747198
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.942032
Daily value at risk   -0.022492
dtype: float64


###8.1.4 Agen SAC

In [91]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return          0.0
Cumulative returns     0.0
Annual volatility      0.0
Sharpe ratio           NaN
Calmar ratio           NaN
Stability              0.0
Max drawdown           0.0
Omega ratio            NaN
Sortino ratio          NaN
Skew                   NaN
Kurtosis               NaN
Tail ratio             NaN
Daily value at risk    0.0
dtype: float64


###8.1.5 Agen DDPG

In [92]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return          0.199258
Cumulative returns     0.428917
Annual volatility      0.350534
Sharpe ratio           0.691851
Calmar ratio           0.797052
Stability              0.424717
Max drawdown          -0.249994
Omega ratio            1.201753
Sortino ratio          1.162522
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.293329
Daily value at risk   -0.043201
dtype: float64
